In [1]:
import sys
sys.path.append('/Users/robert/dev/personal/e2e_autopilot/src')

import vehicle_detect_utils
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import sigmoid
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
Xs, Ys = vehicle_detect_utils.load_data()
test_size = .2
train_xs, test_xs, train_ys, test_ys = train_test_split(Xs, Ys, test_size=test_size)

100%|██████████| 8968/8968 [00:10<00:00, 881.83it/s] 


In [3]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.3,
    horizontal_flip=True)

train_generator.fit(train_xs)


In [4]:
model = Sequential()
model.add(Conv2D(32, 3, padding='same', input_shape=(16, 16, 3)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(LeakyReLU(0.3))
model.add(Dense(1, activation=sigmoid))

model.compile(optimizer=keras.optimizers.Adam(lr=1e-4 / 2), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1048704   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 1,049,729
Trainable params: 1,049,729
Non-trainable params: 0
_________________________________________________________________


In [5]:
epochs = 20
batch_size = 32

model.fit_generator(train_generator.flow(train_xs,
                                         train_ys,
                                         batch_size=batch_size),
                    steps_per_epoch=len(train_xs) // batch_size,
                    epochs=epochs,
                    validation_data=(test_xs, test_ys))

model.evaluate(test_xs, test_ys)


Epoch 1/20
444/444 [==============================] - 16s - loss: 0.4625 - acc: 0.8053 - val_loss: 2.0894 - val_acc: 0.8643
Epoch 2/20
444/444 [==============================] - 16s - loss: 0.3738 - acc: 0.8544 - val_loss: 2.0777 - val_acc: 0.8674
Epoch 3/20
444/444 [==============================] - 16s - loss: 0.3349 - acc: 0.8726 - val_loss: 1.9501 - val_acc: 0.8756
Epoch 4/20
444/444 [==============================] - 16s - loss: 0.3094 - acc: 0.8801 - val_loss: 1.9484 - val_acc: 0.8753
Epoch 5/20
444/444 [==============================] - 16s - loss: 0.2864 - acc: 0.8900 - val_loss: 1.4743 - val_acc: 0.9051
Epoch 6/20
444/444 [==============================] - 16s - loss: 0.2737 - acc: 0.8935 - val_loss: 1.4153 - val_acc: 0.9091
Epoch 7/20
444/444 [==============================] - 16s - loss: 0.2576 - acc: 0.9034 - val_loss: 1.3912 - val_acc: 0.9116
Epoch 8/20
444/444 [==============================] - 16s - loss: 0.2426 - acc: 0.9082 - val_loss: 1.2775 - val_acc: 0.9178
Epoch 9/

KeyboardInterrupt: 

In [6]:
model_json = model.to_json()
with open("/Users/robert/dev/personal/e2e_autopilot/src/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/Users/robert/dev/personal/e2e_autopilot/src/model.h5")
print("Saved model to disk")

Saved model to disk
